In [1]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [ ]:
import openai
import re
import httpx
import os

from openai import OpenAI
client = OpenAI()

MODEL = "gpt-4o-mini"
prompt = "Write something short but funny"

chat_completion = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": prompt}]
)

print(chat_completion.choices[0].message.content)


I told my computer I needed a break, and now it won't stop sending me pop-up ads for tropical vacations!


In [5]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []

        if self.system:
            self.messages.append({"role": "system", "content": system})

    
    def __call__(self, prompt):
        self.messages.append({"role": "user", "content": prompt})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self, model="gpt-4o-mini", temperature=0):
        completion = client.chat.completions.create(
            model=model,
            temperature=temperature,
            messages=self.messages
        )
        return completion.choices[0].message.content

In [6]:
prompt = """
You run in a loop if Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate 4 & 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.

get_cost:
e.g. get_cost: book
returns the cost of a book

wikipedia:
e.g. wikipedia: LangChain
Returns a summary from searching Wikipedia

Always look things up on Wikipedia if you have the opportunity to do so.

Example session #1:

Question: How much does a pen cost?
Thought: I should look the pen cost using get_cost
Action: get_cost: pen
PAUSE

You will be called again with this:

Observation: A pen costs $5

You then outpit:

Answer: A pen costs $5



Example session #2

Question: What is the capital of France?
Thought: I should look up France on Wikipedia
Action: wikipedia: France
PAUSE

You will be called again with this:

Observation: France is a country. The capital is Paris.

You then output:

Answer: The capital of France is Paris
""".strip()

In [9]:
def calculate(what):
    return eval(what)


def get_cost(thing):
    if thing in "pen":
        return ("A pen costs $5")
    elif thing in "book":
        return ("A book costs $20")
    elif thing in "stapler":
        return ("A stapler costs $10")
    else:
        return ("A random thing for writing costs $12")


def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })
    results = response.json().get("query").get("search", [])

    if not results:
        return None
    return results[0]["snippet"]

In [ ]:
wikipedia("langchain")

'Free and open-source software portal <span class="searchmatch">LangChain</span> is a software framework that helps facilitate the integration of large language models (LLMs) into applications'

In [11]:
known_actions = {
    "calculate": calculate,
    "get_cost": get_cost,
    "wikipedia": wikipedia
}

In [15]:
my_agent = Agent(prompt)
result = my_agent("How much does a pen cost?")

print(result)

Thought: I should look up the cost of a pen using the get_cost action.  
Action: get_cost: pen  
PAUSE


In [20]:
next_prompt = f"Obseravation: {get_cost('pen')}"
next_prompt

'Obseravation: A pen costs $5'

In [21]:
my_agent(next_prompt)

'Answer: A pen costs $5.'

In [22]:
my_agent.messages

[{'role': 'system',
  'content': 'You run in a loop if Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate 4 & 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then outpit:\n\nAnswer:

In [23]:
abot = Agent(prompt)

question = "I want to buy a pen and a book. How much do they cost in total?"

abot(question)

"Thought: I need to find the cost of both a pen and a book to calculate the total cost. I'll start by looking up the cost of a pen. \nAction: get_cost: pen\nPAUSE"

In [25]:
next_prompt = f"Observation: {get_cost('pen')}"
print(next_prompt)

Observation: A pen costs $5


In [26]:
abot(next_prompt)

'Thought: Now that I have the cost of the pen, I need to find the cost of a book to calculate the total. \nAction: get_cost: book\nPAUSE'

In [27]:
next_prompt = f"Observation: {get_cost('book')}"
print(next_prompt)

Observation: A book costs $20


In [28]:
abot(next_prompt)

'Thought: I now have the costs of both the pen and the book. The pen costs $5 and the book costs $20. I will calculate the total cost by adding these two amounts together. \nAction: calculate: 5 + 20\nPAUSE'

In [29]:
next_prompt = f"Observation: {calculate('5 + 20')}"
print(next_prompt)

Observation: 25


In [30]:
abot(next_prompt)

'Answer: The total cost of a pen and a book is $25.'

In [31]:
abot = Agent(prompt)

query = "2024 United Kingdom elections"

abot(query)

'Thought: I should look up information about the 2024 United Kingdom elections on Wikipedia to get the latest details.  \nAction: wikipedia: 2024 United Kingdom elections  \nPAUSE'

In [32]:
next_prompt = f"Observation: {wikipedia(query)}"
print(next_prompt)

Observation: The <span class="searchmatch">2024</span> <span class="searchmatch">United</span> <span class="searchmatch">Kingdom</span> general <span class="searchmatch">election</span> was held on Thursday, 4 July <span class="searchmatch">2024</span> to elect all 650 members of the House of Commons. The opposition Labour Party


In [33]:
abot(next_prompt)

'Answer: The 2024 United Kingdom general election is scheduled to be held on Thursday, 4 July 2024, to elect all 650 members of the House of Commons.'

In [34]:
abot.messages

[{'role': 'system',
  'content': 'You run in a loop if Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate 4 & 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then outpit:\n\nAnswer:

In [ ]:
[{'role': 'system',
  'content': 'You run in a loop if Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate 4 & 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then outpit:\n\nAnswer: A pen costs $5\n\n\n\nExample session #2\n\nQuestion: What is the capital of France?\nThought: I should look up France on Wikipedia\nAction: wikipedia: France\nPAUSE\n\nYou will be called again with this:\n\nObservation: France is a country. The capital is Paris.\n\nYou then output:\n\nAnswer: The capital of France is Paris'},
 {'role': 'user', 'content': '2024 United Kingdom elections'},
 {'role': 'assistant',
  'content': 'Thought: I should look up information about the 2024 United Kingdom elections on Wikipedia to get the latest details.  \nAction: wikipedia: 2024 United Kingdom elections  \nPAUSE'},
 {'role': 'user',
  'content': 'Observation: The <span class="searchmatch">2024</span> <span class="searchmatch">United</span> <span class="searchmatch">Kingdom</span> general <span class="searchmatch">election</span> was held on Thursday, 4 July <span class="searchmatch">2024</span> to elect all 650 members of the House of Commons. The opposition Labour Party'},
 {'role': 'assistant',
  'content': 'Answer: The 2024 United Kingdom general election is scheduled to be held on Thursday, 4 July 2024, to elect all 650 members of the House of Commons.'}]

One of the be

[{'role': 'system',
  'content': 'You run in a loop if Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer.\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate 4 & 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary.\n\nget_cost:\ne.g. get_cost: book\nreturns the cost of a book\n\nwikipedia:\ne.g. wikipedia: LangChain\nReturns a summary from searching Wikipedia\n\nAlways look things up on Wikipedia if you have the opportunity to do so.\n\nExample session #1:\n\nQuestion: How much does a pen cost?\nThought: I should look the pen cost using get_cost\nAction: get_cost: pen\nPAUSE\n\nYou will be called again with this:\n\nObservation: A pen costs $5\n\nYou then outpit:\n\nAnswer:

In [51]:
def calculate(what):
    return eval(what)


# 2) Make get_cost robust
def get_cost(thing):
    t = thing.strip().lower()
    # handle simple plurals
    if t.endswith('s'):
        t = t[:-1]
    if t == "pen":
        return "A pen costs $5"
    elif t == "book":
        return "A book costs $20"
    elif t == "stapler":
        return "A stapler costs $10"
    else:
        return "A random thing for writing costs $12"


def wikipedia(q):
    response = httpx.get("https://en.wikipedia.org/w/api.php", params={
        "action": "query",
        "list": "search",
        "srsearch": q,
        "format": "json"
    })
    results = response.json().get("query").get("search", [])

    if not results:
        return None
    return results[0]["snippet"]

In [55]:
# 1) Tighten the action regex to ignore extra spaces
action_re = re.compile(r'^Action:\s*(\w+):\s*(.+?)\s*$')

def query(question, max_turns=5):
    i = 0
    bot = Agent(prompt)
    next_prompt = question

    while i < max_turns:
        i += 1
        result = bot(next_prompt)
        print(result)

        actions = [action_re.match(a) for a in result.split("\n") if action_re.match(a)]
        if actions:
            action, action_input = actions[0].groups()
            action = action.strip().lower()
            action_input = action_input.strip()         # <-- strip here

            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")

            print(f" -- running {action} {action_input!r}")
            observation = known_actions[action](action_input)
            print(f"Observation: {observation}")
            next_prompt = f"Observation: {observation}"
        else:
            return

In [56]:
question = """I want to buy 2 books and 3 pens. How much do I have to pay?"""
query(question)

Thought: I need to find the cost of both a book and a pen to calculate the total cost for 2 books and 3 pens. I will first look up the cost of a book.  
Action: get_cost: book  
PAUSE  
 -- running get_cost 'book'
Observation: A book costs $20
Thought: Now that I know the cost of a book is $20, I need to find the cost of a pen to complete the calculation.  
Action: get_cost: pen  
PAUSE  
 -- running get_cost 'pen'
Observation: A pen costs $5
Thought: I have the costs now: a book costs $20 and a pen costs $5. I will calculate the total cost for 2 books and 3 pens.  
Action: calculate: 2 * 20 + 3 * 5  
PAUSE  
 -- running calculate '2 * 20 + 3 * 5'
Observation: 55
Answer: You have to pay $55 for 2 books and 3 pens.


In [ ]:
question = """UEFA EURO 2024"""
query(question)

Thought: I should look up UEFA EURO 2024 on Wikipedia to gather information about the event.  
Action: wikipedia: UEFA EURO 2024  
PAUSE  
 -- running wikipedia 'UEFA EURO 2024'
Observation: The <span class="searchmatch">2024</span> <span class="searchmatch">UEFA</span> Men&#039;s European Football Championship, commonly referred to as <span class="searchmatch">UEFA</span> <span class="searchmatch">Euro</span> <span class="searchmatch">2024</span> (stylised as <span class="searchmatch">UEFA</span> <span class="searchmatch">EURO</span> <span class="searchmatch">2024</span>) or simply <span class="searchmatch">Euro</span> <span class="searchmatch">2024</span>, was the 17th
Answer: UEFA EURO 2024 is the 17th UEFA Men's European Football Championship.
